<h1 align="center"><font color="yellow">LangChain 4: Bate-papo em LangChain</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

In [1]:
#%pip install -qU langchain openai

Note: you may need to restart the kernel to use updated packages.


Começaremos inicializando o objeto `ChatOpenAI`. Para isso, precisaremos de uma `chave de API OpenAI`. Observe que há naturalmente um pequeno custo para executar este notebook devido à natureza paga do acesso à `API do OpenAI`.

In [8]:
# Isto é quando usas o arquivo .env:
from dotenv import load_dotenv
import os
print('Carregando a minha chave Key: ', load_dotenv())
Eddy_API_KEY_OpenAI = os.environ['OPENAI_API_KEY'] 


Carregando a minha chave Key:  True


<font color="orange">Inicialize o objeto `ChatOpenAI`. Definiremos a `temperature = 0` para minimizar a aleatoriedade e tornar as `saídas repetíveis`.</font>

In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    openai_api_key=Eddy_API_KEY_OpenAI,
    temperature=0,
    model='gpt-3.5-turbo'
)


<font color="orange">Os chats com o modelo `Chat-GPT gpt-3.5-turbo` são normalmente estruturados da seguinte forma:</font>


```
System: Você é um assistente útil.

User: Oi AI, como você está hoje?

Assistant: Estou ótimo obrigado. Como posso ajudá-lo?

User: Eu gostaria de entender a teoria das cordas.

Assistant: 
```



O "Assistant:" final sem uma resposta é o que levaria o modelo a continuar a conversação. No endpoint oficial da `OpenAI ChatCompletion`, eles seriam passados para o modelo em um formato como:



```
[
    {"role": "system", "content": "Você é um assistente útil."},
    {"role": "user", "content": "Oi AI, como você está hoje?"},
    {"role": "assistant", "content": "Estou ótimo obrigado. Como posso ajudá-lo?"}
    {"role": "user", "content": "Eu gostaria de entender a teoria das cordas."}
]
```


<font color="orange">No `LangChain` existe um formato ligeiramente diferente. Usamos três objetos de mensagem da seguinte forma:</font>

In [3]:
from langchain.schema import (SystemMessage, HumanMessage, AIMessage)



messages = [
    SystemMessage(content="Você é um assistente útil."),
    HumanMessage(content="Oi AI, como você está hoje?"),
    AIMessage(content="Estou ótimo obrigado. Como posso ajudá-lo?"),
    HumanMessage(content="Eu gostaria de entender a teoria das cordas.")
]



<font color="orange">O formato é muito parecido, apenas trocamos a função de `"user"` por `HumanMessage`, e a função de `"Assistant"` por `AIMessage`.

Geramos a próxima resposta da AI passando essas mensagens para o objeto `ChatOpenAI`.</font>

In [5]:
res = chat(messages)
res


AIMessage(content='A teoria das cordas é uma teoria física que tenta unificar a relatividade geral e a mecânica quântica. Ela propõe que as partículas subatômicas não são pontos, mas sim cordas vibrantes em um espaço-tempo de 10 ou 11 dimensões. Essas cordas podem vibrar em diferentes frequências, o que determina suas propriedades, como massa e carga elétrica.\n\nA teoria das cordas é uma das principais candidatas a uma teoria de tudo, que busca explicar todas as forças fundamentais da natureza em um único quadro teórico. No entanto, ainda há muitas questões em aberto e a teoria das cordas continua sendo objeto de intensa pesquisa e debate na comunidade científica.', additional_kwargs={})

<font color="orange">Em resposta, obtemos outro `objeto de mensagem AI`. Podemos imprimi-lo mais claramente assim:</font>

In [6]:
print(res.content)


A teoria das cordas é uma teoria física que tenta unificar a relatividade geral e a mecânica quântica. Ela propõe que as partículas subatômicas não são pontos, mas sim cordas vibrantes em um espaço-tempo de 10 ou 11 dimensões. Essas cordas podem vibrar em diferentes frequências, o que determina suas propriedades, como massa e carga elétrica.

A teoria das cordas é uma das principais candidatas a uma teoria de tudo, que busca explicar todas as forças fundamentais da natureza em um único quadro teórico. No entanto, ainda há muitas questões em aberto e a teoria das cordas continua sendo objeto de intensa pesquisa e debate na comunidade científica.


<font color="orange">Como `res` é apenas outro objeto `AIMessage`, podemos anexá-lo às `messages`, adicionar outro `HumanMessage` e gerar a próxima resposta na conversa.</font>

In [ ]:
# Adiciona a última resposta de IA às mensagens
messages.append(res)


# Criamos um novo PROMPT de usuário:
prompt = HumanMessage(
    content="Por que os físicos acreditam que ela pode produzir uma 'teoria unificada'?"
)


# Adicionar às mensagens
messages.append(prompt)

# Enviar para Chat-GPT:
res = chat(messages)

print(res.content)


# Novos modelos de prompt (New Prompt Templates)

Juntamente com o que vimos até agora, também existem `três` novos modelos de prompt que podemos usar. Esses são `SystemMessagePromptTemplate`, `AIMessagePromptTemplate` e `HumanMessagePromptTemplate`.


Estes são simplesmente uma extensão dos [Templates de prompt do Langchain](https://www.pinecone.io/learn/langchain-prompt-templates/) que modificam o `"prompt"` de retorno para ser um objeto `SystemMessage`, `AIMessage` ou `HumanMessage`, respectivamente.

Por enquanto, não há um grande número de casos de uso para esses objetos. No entanto, se tivermos algo que gostaríamos de adicionar às nossas mensagens, isso pode ser útil. `Por exemplo`, digamos que gostaríamos que nossas respostas de `IA` sempre consistissem em não mais que `50` caracteres.

Usando o `modelo OpenAI gpt-3.5-turbo-0301` atual, podemos ter problemas se passarmos esta instrução apenas na primeira mensagem do sistema.

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (SystemMessage, HumanMessage, AIMessage)


chat = ChatOpenAI(
    openai_api_key=Eddy_API_KEY_OpenAI,
    temperature=0,
    model='gpt-3.5-turbo-0301'
)


# Configuramos primeira mensagem do sistema
messages = [
    SystemMessage(content=(
        'Você é um assistente útil. Você mantém respostas para não mais do que '
        '100 caracteres (incluindo espaços em branco) e assine cada '
        'mensagem com um nome aleatório como "Robot McRobot" ou "Bot Rob".'
    )),
    HumanMessage(content="Olá IA, como vai? O que é física quântica?")
]


<font color="orange">Agora usamos, completion:</font>

In [6]:
res = chat(messages)

print(f"Length: {len(res.content)}\n{res.content}")


Length: 229
Olá! A física quântica é o estudo do comportamento da matéria e da energia em escalas muito pequenas, como átomos e partículas subatômicas. Ela descreve fenômenos como superposição, emaranhamento e tunelamento quântico. - Bot Rob


<font color="orange">Ok, parece que nosso assistente de IA não é tão bom em seguir nenhuma de nossas instruções. E se adicionarmos essas instruções ao `HumanMessage` por meio de um `HumanMessagePromptTemplate`?</font>

In [5]:
from langchain.prompts.chat import HumanMessagePromptTemplate, ChatPromptTemplate

human_template = HumanMessagePromptTemplate.from_template(
    '{input} Você pode manter a resposta em até 100 caracteres '+
    '(incluindo espaço em branco) e assine com um nome aleatório como "Robô" '+
    'McRobot" ou "Bot Rob".'
)


# Criamos a mensagem humana
chat_prompt = ChatPromptTemplate.from_messages([human_template])
# Formatar com alguma entrada
chat_prompt_value = chat_prompt.format_prompt(
    input="Olá IA, como vai? O que é física quântica?"
)


chat_prompt_value

ChatPromptValue(messages=[HumanMessage(content='Olá IA, como vai? O que é física quântica? Você pode manter a resposta em até 100 caracteres (incluindo espaço em branco) e assine com um nome aleatório como "Robô" McRobot" ou "Bot Rob".', additional_kwargs={})])

Observe que, para usar `HumanMessagePromptTemplate` como um modelo de prompt típico com o método `.format_prompt`, precisamos passá-lo por um objeto `ChatPromptTemplate`. Esse é o caso de todos os novos modelos de prompt baseados em bate-papo.

Usando isso, retornamos um objeto `ChatPromptValue`. Isso pode ser formatado em uma lista ou string da seguinte forma:

In [8]:
chat_prompt_value.to_messages()

[HumanMessage(content='Olá IA, como vai? O que é física quântica? Você pode manter a resposta em até 100 caracteres (incluindo espaço em branco) e assine com um nome aleatório como "Robô" McRobot" ou "Bot Rob".', additional_kwargs={})]

In [9]:
chat_prompt_value.to_string()

'Human: Olá IA, como vai? O que é física quântica? Você pode manter a resposta em até 100 caracteres (incluindo espaço em branco) e assine com um nome aleatório como "Robô" McRobot" ou "Bot Rob".'

<font color="orange">Vamos ver se essa nova abordagem funciona.</font>

In [ ]:
messages = [
    SystemMessage(content=(
        ' Você pode manter a resposta em até 100 caracteres '
        '(incluindo espaço em branco) e assine com um nome aleatório como "Robô" '
        'McRobot" ou "Bot Rob".'
    )),
    chat_prompt.format_prompt(
        input="Olá IA, como vai? O que é física quântica?"
    ).to_messages()[0]
]

res = chat(messages)

print(f"Length: {len(res.content)}\n{res.content}")